In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
harmony_data = pd.read_csv(r'C:\Users\USUARIO\Documents\GitHub\Oil-Gas-Projects\Reservoir Engineering\Datasets\Oil Production\Harmony_data.csv')
IHS_data = pd.read_csv(r'C:\Users\USUARIO\Documents\GitHub\Oil-Gas-Projects\Reservoir Engineering\Datasets\Oil Production\IHS_data.csv')
train_data = pd.read_csv(r'C:\Users\USUARIO\Documents\GitHub\Oil-Gas-Projects\Reservoir Engineering\Datasets\Oil Production\production_data_train.csv')
test_data = pd.read_csv(r'C:\Users\USUARIO\Documents\GitHub\Oil-Gas-Projects\Reservoir Engineering\Datasets\Oil Production\production_data_test.csv')
test_APIs = pd.read_csv(r'C:\Users\USUARIO\Documents\GitHub\Oil-Gas-Projects\Reservoir Engineering\Datasets\Oil Production\Test_APIs.csv')
test_APIs.shape, harmony_data.shape, IHS_data.shape, train_data.shape, test_data.shape,

((1134, 1), (8930, 7), (9224, 14), (394074, 10), (6489, 11))

In [15]:
harmony_data.head()

,API,LATERAL_LENGTH_BLEND,PROP_PER_FOOT,WATER_PER_FOOT,GOR_30,GOR_60,GOR_90
0,5123371740000,9056,1025.8680,790.7205,2300.919963,2290.231375,2171.635752
1,5123377360000,4101,NaN,NaN,3786.669639,3838.096579,4012.142621
2,5123378210000,4161,916.0801,839.9280,2399.490505,2291.864862,2472.275585
3,5123378650100,4129,831.1531,NaN,3907.549121,4122.576934,4553.377380
4,5123380130000,4539,1032.2330,755.1343,1758.110773,1323.960532,1182.969280


In [3]:
train_data.head()

,API,Year,Month,Liquid,Gas,RatioGasOil,Water,PercentWater,DaysOn,_LastUpdate
0,33053063590000,2015,3,17186,26624,1.549168,8213,0.477889,16,2016-08-11 14:03:14.000
1,33053063590000,2015,4,18796,21796,1.159608,7255,0.385986,30,2016-08-11 14:03:14.000
2,33053063590000,2015,5,14626,15988,1.093122,4213,0.288049,31,2016-08-11 14:03:14.000
3,33053063590000,2015,6,10421,12311,1.181365,2717,0.260724,30,2016-04-06 13:47:11.383
4,33053063590000,2015,7,9777,11254,1.151069,2399,0.245372,31,2016-04-06 13:47:11.383


In [4]:
# Exploratory Data Analysis
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 394074 entries, 0 to 394073
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   API           394074 non-null  int64  
 1   Year          394074 non-null  int64  
 2   Month         394074 non-null  int64  
 3   Liquid        394074 non-null  int64  
 4   Gas           394074 non-null  int64  
 5   RatioGasOil   394074 non-null  float64
 6   Water         394074 non-null  int64  
 7   PercentWater  394074 non-null  float64
 8   DaysOn        394074 non-null  int64  
 9   _LastUpdate   394074 non-null  object 
dtypes: float64(2), int64(7), object(1)
memory usage: 30.1+ MB


In [5]:
# Check for missing values
train_data.isnull().sum()

API             0
Year            0
Month           0
Liquid          0
Gas             0
RatioGasOil     0
Water           0
PercentWater    0
DaysOn          0
_LastUpdate     0
dtype: int64

In [6]:
# Compare the number of observations in other datasets
IHS_data.API.unique().shape

(8930,)

In [7]:
# Compare the number of observations in other datasets
harmony_data.API.unique().shape

(8930,)

In [8]:
# Get repetead wells in train and test data
repetead_wells = []
for well in train_data.API.unique():
    if well in test_data.API.unique():
        repetead_wells.append(well)

print(len(repetead_wells))

43


#### Check values of each column

In [9]:
# 
train_data[train_data.API.isin(repetead_wells)].shape

(2145, 10)

In [10]:
# Compare the same WELL in train and test data
train_data[(train_data['API'] == 42301322730000) & (train_data['Year'] == 2015)]

,API,Year,Month,Liquid,Gas,RatioGasOil,Water,PercentWater,DaysOn,_LastUpdate
393204,42301322730000,2015,2,10801,36697,3.397556,31584,2.924174,0,2018-11-04 10:19:16.577
393205,42301322730000,2015,3,17497,78231,4.471109,67331,3.848145,0,2018-11-04 10:19:16.577
393206,42301322730000,2015,4,12283,65415,5.325653,56301,4.583652,0,2018-11-04 10:19:16.577
393207,42301322730000,2015,5,9661,64795,6.706863,55767,5.772384,0,2018-11-04 10:19:16.577
393208,42301322730000,2015,6,8056,53236,6.608242,45819,5.687562,0,2018-11-04 10:19:16.577
393209,42301322730000,2015,7,7166,53470,7.461624,46020,6.421993,0,2018-11-04 10:19:16.577
393210,42301322730000,2015,8,6502,76269,11.730083,65642,10.095663,0,2018-11-04 10:19:16.577
393211,42301322730000,2015,9,5658,53272,9.415341,45850,8.103570,0,2018-11-04 10:19:16.577
393212,42301322730000,2015,10,5395,51993,9.637257,44749,8.294532,0,2018-11-04 10:19:16.577
393213,42301322730000,2015,11,4885,45718,9.358854,39348,8.054862,0,2018-11-04 10:19:16.577


In [11]:
# Compare the same WELL in train and test data
test_data[(test_data['API'] == 42301322730000)]

,API,Year,Month,Liquid,Gas,RatioGasOil,Water,PercentWater,DaysOn,_LastUpdate,date
6478,42301322730000,2015,2,10801,36697,3.397556,31584,2.924174,0,2018-11-04 10:19:16.577,2015-02-01
6479,42301322730000,2015,3,17497,78231,4.471109,67331,3.848145,0,2018-11-04 10:19:16.577,2015-03-01
6480,42301322730000,2015,4,12283,65415,5.325653,56301,4.583652,0,2018-11-04 10:19:16.577,2015-04-01
6481,42301322730000,2015,5,9661,64795,6.706863,55767,5.772384,0,2018-11-04 10:19:16.577,2015-05-01


In [12]:
test_APIs.API.unique().shape

(1130,)

In [13]:
test_data.index.unique().shape

(6489,)

In [14]:
set(test_APIs.index.unique()) - set(test_data.index.unique())

set()